# Яндекс.Музыка

**Цель исследования** — проверка трех основных гипотез:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Исследование пройдёт в три основных этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных

Начнем с загрузки необходимых библиотек.

In [5]:
import pandas as pd

Теперь загрузим исходные данные, а также выведем основную информацию о них. 

In [6]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

In [7]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Найдите ещё одну проблему в названии колонок и опишите её в этом пункте.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Стиль заголовков

Предобработку данных начнем с корректировки заголовков столбцов. Для начала выведем исходные варианты заголовков. 

In [18]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с хорошим стилем:
* перепишем названия в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

In [19]:
df = df.rename(columns = {'  userID' : 'user_id', 'Track' : 'track', '  City  ' : 'city', 'Day' : 'day'})

Проверим результат, повторно выведя на экран названия столбцов.

In [20]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений
Перейдем к обработке пропусков. Для начала выведем число пропусков в каждой из переменных.

In [21]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски присутствуют в трех переменных. В `track` и `artist` пропуски не важны для вашей работы, а значит достаточно поставить на месте пропусков заглушку.

В то же время пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Однако, посольку восстановление потеряных данных для нас не представляется возможным, также используем заглушку.

In [22]:
for name in ['track', 'artist', 'genre']:
    df[name] = df[name].fillna('unknown')

Убедимся, что в таблице не осталось пропусков.

In [23]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты
Переходим к следующему блоку предобработки - анализу выборки на дубликаты. Начнем с выявления явных дубликатов.

In [24]:
display(f'Количество пропусков {df.duplicated().sum()}.')

'Количество пропусков 3826.'

Удалим выявленные дубликаты, чтобы они не влияли на результаты анализа. 

In [25]:
df = df.drop_duplicates().reset_index(drop=True)

Убедимся, что полностью избавились от явных дубликатов.

In [26]:
display(f'Количество пропусков {df.duplicated().sum()}.')

'Количество пропусков 0.'

Теперь избавимся от неявных дубликатов в колонке `genre`. Они могли возникнуть в тех случаях, если название одного и того же жанра было записано по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. 

In [27]:
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


Можно заметить следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Приведем их к общему названию.

In [28]:
# Функция для замены неявных дубликатов

def replace_wrong_genres(wrong_genres, correct_genre):
    for mistake in wrong_genres:
        df['genre'] = df['genre'].replace(mistake, correct_genre)

In [29]:
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')

**Выводы**

На этапе предобработки мы обнаружили и устранили следующие проблемы в данных:
- нарушения стиля заголовков,
- наличие пропущенных значений,
- наличие дубликатов — как явных, так и неявных.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для начала оценим активность пользователей в каждом городе. Сгруппируйте данные по городу и посчитайте прослушивания в каждой группе.



In [40]:
display(df.groupby('city')['track'].count())

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в каждый из доступных дней недели.


In [42]:
display(df.groupby('day')['track'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Напишем функцию, которая будет рассчитывать количество прослушиваний для заданного города и дня недели.

In [43]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Теперь выведем количество прослушиваний для Москвы и Санкт-Петербурга для каждого доступного дня недели. 

In [44]:
msk_mon = number_tracks('Monday', 'Moscow')
display(f'Количество прослушиваний в Москве по понедельникам: {msk_mon}.')

'Количество прослушиваний в Москве по понедельникам: 15740.'

In [45]:
spb_mon = number_tracks('Monday', 'Saint-Petersburg')
display(f'Количество прослушиваний в Санкт-Петербурге по понедельникам: {spb_mon}.')

'Количество прослушиваний в Санкт-Петербурге по понедельникам: 5614.'

In [46]:
msk_wed = number_tracks('Wednesday', 'Moscow')
display(f'Количество прослушиваний в Москве по средам: {msk_wed}.')

'Количество прослушиваний в Москве по средам: 11056.'

In [47]:
spb_wed = number_tracks('Wednesday', 'Saint-Petersburg')
display(f'Количество прослушиваний в Санкт-Петербурге по средам: {spb_wed}.')

'Количество прослушиваний в Санкт-Петербурге по средам: 7003.'

In [48]:
msk_fri = number_tracks('Friday', 'Moscow')
display(f'Количество прослушиваний в Москве по пятницам: {msk_fri}.')

'Количество прослушиваний в Москве по пятницам: 15945.'

In [49]:
spb_fri = number_tracks('Friday', 'Saint-Petersburg')
display(f'Количество прослушиваний в Санкт-Петербурге по пятницам: {spb_fri}.')

'Количество прослушиваний в Санкт-Петербурге по пятницам: 5895.'

Для наглядности соединим данные в общую таблицу.

In [50]:
pd.DataFrame(data = [['Moscow', msk_mon, msk_wed, msk_fri], ['Saint-Petersburg', spb_mon, spb_wed, spb_fri]], columns = ['city', 'monday', 'wednesday', 'friday'])

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу между поведением пользователей:
- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Для удобства анализа разделим исходные данные - для каждого из городов в отдельности. 

In [51]:
moscow_general = df[df['city'] == 'Moscow']

In [52]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Для проверки второй гипотезы создадим функцию, которая будет возвращать информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [53]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted['genre'].head(10)

Сравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [54]:
print('Прослушивания по жанрам в утро понедельника в Москве:')
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

Прослушивания по жанрам в утро понедельника в Москве:


genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [55]:
print('Прослушивания по жанрам в утро понедельника в Санкт-Петербурге:')
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

Прослушивания по жанрам в утро понедельника в Санкт-Петербурге:


genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [56]:
print('Прослушивания по жанрам в вечер пятницы в Москве:')
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

Прослушивания по жанрам в вечер пятницы в Москве:


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

In [57]:
print('Прослушивания по жанрам в вечер пятницы в Санкт-Петербурге:')
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

Прослушивания по жанрам в вечер пятницы в Санкт-Петербурге:


genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:
1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.
2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра.

In [58]:
moscow_general = moscow_general.groupby('genre')
moscow_genres = moscow_general['user_id'].count().sort_values(ascending = False)

Выведите на экран первые десять строк `moscow_genres`:

In [59]:
moscow_genres[:10]

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: user_id, dtype: int64

Теперь повторим то же и для Петербурга.

In [60]:
spb_general = spb_general.groupby('genre')
spb_genres = spb_general['user_id'].count().sort_values(ascending = False)

In [61]:
spb_genres[:10]

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: user_id, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Перед нами стояла задача проверки трех гипотез на реальных данных Яндекс.Музыки. В ходе работы над проектом были установлены следующие выводы:
1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 